In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
movies = pd.read_csv('../data/preprocessed/movies_id_updated.csv')

In [3]:
movies

,id,title,imdbID,spanishTitle,imdbPictureURL,year,rtID,rtPictureURL
0,1,Toy story,tt0114709,Toy story (juguetes),http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995.0,toy_story,http://content7.flixster.com/movie/10/93/63/10...
1,2,Jumanji,tt0113497,Jumanji,http://ia.media-imdb.com/images/M/MV5BMzM5NjE1...,1995.0,1068044-jumanji,http://content8.flixster.com/movie/56/79/73/56...
2,3,Grumpy Old Men,tt0107050,Dos viejos gruñones,http://ia.media-imdb.com/images/M/MV5BMTI5MTgy...,1993.0,grumpy_old_men,http://content6.flixster.com/movie/25/60/25602...
3,4,Waiting to Exhale,tt0114885,Esperando un respiro,http://ia.media-imdb.com/images/M/MV5BMTczMTMy...,1995.0,waiting_to_exhale,http://content9.flixster.com/movie/10/94/17/10...
4,5,Father of the Bride Part II,tt0113041,Vuelve el padre de la novia (Ahora también abu...,http://ia.media-imdb.com/images/M/MV5BMTg1NDc2...,1995.0,father_of_the_bride_part_ii,http://content8.flixster.com/movie/25/54/25542...
...,...,...,...,...,...,...,...,...
10192,65088,Bedtime Stories,tt0960731,Más allá de los sueños,http://ia.media-imdb.com/images/M/MV5BMjA5Njk5...,2008.0,bedtime_stories,http://content6.flixster.com/movie/10/94/33/10...
10193,65091,Manhattan Melodrama,tt0025464,El enemigo público número 1,http://ia.media-imdb.com/images/M/MV5BMTUyODE3...,1934.0,manhattan_melodrama,http://content9.flixster.com/movie/66/44/64/66...
10194,65126,Choke,tt1024715,Choke,http://ia.media-imdb.com/images/M/MV5BMTMxMDI4...,2008.0,choke,http://content6.flixster.com/movie/10/85/09/10...
10195,65130,Revolutionary Road,tt0959337,Revolutionary Road,http://ia.media-imdb.com/images/M/MV5BMTI2MzY2...,2008.0,revolutionary_road,http://content8.flixster.com/movie/10/88/40/10...


In [4]:
def get_features(api_key, ID):
    """
    api_key: key for accessing the API
    ID: selects which slice of IDs to download
    """
    omdb_features = pd.DataFrame()    
    counter = 0
    # iterate over each 900 imdb IDs, specified by param ID
    for i in movies['imdbID'][ID*900:(ID+1)*900]:
        try:
            # make request to omdb API
            req = requests.get('http://www.omdbapi.com/?apikey='+api_key+'&i='+i)
            # convert into json
            data = req.json()
            # convert into dataframe
            feature = pd.DataFrame(data)
            # Several ratings are available in column 'Ratings' (max 3) which creates a row for each rating: Select each rating, create individual column and add to feature dataframe 
            dummy = pd.DataFrame(feature['Ratings'][0], index=[0]).pivot(columns='Source', values='Value')
            for k in range(1,feature.shape[0]):
                dummy = dummy.join(pd.DataFrame(feature['Ratings'][k], index=[0]).pivot(columns='Source', values='Value'))

            feature = feature.head(1).join(dummy)
            # delete unnecessary column
            del feature['Ratings']
            # append to large df
            omdb_features = omdb_features.append(feature)
            counter+=1
            if counter % 100 == 0 :
                print('number of retrieved IDs: ', counter)
        except ValueError:
            print('No entry found for ID ',i)
            counter+=1

    # rename columns
    feature.rename(columns={'Internet Movie Database':'Rating_IMDB', 'Rotten Tomatoes': 'Rating_Rotten_Tomatoes', 'Metacritic':'Rating_Metacritic'})
    # save to csv, according to which part of the IDs
    omdb_features.to_csv('../data/raw/omdb_'+str(ID)+'.csv')

Insert your api key and run one cell a day

Max: ID 0-2

In [42]:
get_features(api_key='57aa589f', ID=0)

number of retrieved IDs:  100
number of retrieved IDs:  200
number of retrieved IDs:  300
number of retrieved IDs:  400
number of retrieved IDs:  500
number of retrieved IDs:  600
number of retrieved IDs:  700
number of retrieved IDs:  800
number of retrieved IDs:  900


In [5]:
get_features(api_key = '57aa589f', ID=1)

number of retrieved IDs:  100
number of retrieved IDs:  200
number of retrieved IDs:  300
number of retrieved IDs:  400
number of retrieved IDs:  500
number of retrieved IDs:  600
number of retrieved IDs:  700
number of retrieved IDs:  800
number of retrieved IDs:  900


In [10]:
get_features(api_key = '694aa939', ID=2)

number of retrieved IDs:  100
number of retrieved IDs:  200
number of retrieved IDs:  300
No entry found for ID  tt0800175
number of retrieved IDs:  400
number of retrieved IDs:  500
number of retrieved IDs:  600
number of retrieved IDs:  700
number of retrieved IDs:  800


In [5]:
get_features(api_key = 'e1382141', ID=3, sort='True')

/Users/FlorianGrabe/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


number of retrieved IDs:  100
number of retrieved IDs:  200
number of retrieved IDs:  300
number of retrieved IDs:  400
number of retrieved IDs:  500
number of retrieved IDs:  600
number of retrieved IDs:  700
number of retrieved IDs:  800
number of retrieved IDs:  900


In [12]:
get_features(api_key = 'e1382141', ID=4)

number of retrieved IDs:  100
number of retrieved IDs:  200
number of retrieved IDs:  300
number of retrieved IDs:  400
number of retrieved IDs:  500
number of retrieved IDs:  600
number of retrieved IDs:  700
number of retrieved IDs:  800
number of retrieved IDs:  900


In [ ]:
get_features(api_key = 'e1382141', ID=5)

Felix: ID 6-7

In [ ]:
get_features(api_key = '', ID=6)

In [ ]:
get_features(api_key = '', ID=7)

Christin: ID 8-9

In [6]:
get_features(api_key = 'ff8f29d6', ID=8)

number of retrieved IDs:  100
No entry found for ID  tt0054462
number of retrieved IDs:  200
number of retrieved IDs:  300
number of retrieved IDs:  400
number of retrieved IDs:  500
number of retrieved IDs:  600
number of retrieved IDs:  700
number of retrieved IDs:  800
No entry found for ID  tt0439663
number of retrieved IDs:  900


In [7]:
get_features(api_key = 'ff8f29d6', ID=9)

number of retrieved IDs:  100
No entry found for ID  tt0087197
No entry found for ID  tt0087451
No entry found for ID  tt0087968
No entry found for ID  tt0088275
No entry found for ID  tt0088461
No entry found for ID  tt0089264
No entry found for ID  tt0089374
No entry found for ID  tt0089603
No entry found for ID  tt0089869
No entry found for ID  tt0090056
No entry found for ID  tt0090094
No entry found for ID  tt0090270
No entry found for ID  tt0090319
No entry found for ID  tt0090557
No entry found for ID  tt0090713
No entry found for ID  tt0090856
No entry found for ID  tt0091642
No entry found for ID  tt0091670
No entry found for ID  tt0092263
No entry found for ID  tt0093412
No entry found for ID  tt0093777
No entry found for ID  tt0093978
No entry found for ID  tt0258463
No entry found for ID  tt0094888
No entry found for ID  tt0095444
No entry found for ID  tt0096639
No entry found for ID  tt0097814
No entry found for ID  tt0097904
No entry found for ID  tt0099018
No entry foun

No entry found for ID  tt0040458
No entry found for ID  tt0319728
No entry found for ID  tt0086429
No entry found for ID  tt0090037
No entry found for ID  tt0095036
No entry found for ID  tt0020960
No entry found for ID  tt0084173
No entry found for ID  tt0253798
No entry found for ID  tt0279037
No entry found for ID  tt0100606
No entry found for ID  tt0061856
No entry found for ID  tt0104804
No entry found for ID  tt0070246
No entry found for ID  tt0329106
No entry found for ID  tt0326208
No entry found for ID  tt0073107
No entry found for ID  tt0074901
No entry found for ID  tt0045557
No entry found for ID  tt0113649
No entry found for ID  tt0373024
No entry found for ID  tt0034167
No entry found for ID  tt0310203
No entry found for ID  tt0330243
No entry found for ID  tt0098663
No entry found for ID  tt0043255
No entry found for ID  tt0041085
No entry found for ID  tt0352520
No entry found for ID  tt0401233
No entry found for ID  tt0066534
No entry found for ID  tt0076363
No entry f

No entry found for ID  tt0347369
No entry found for ID  tt0370986
No entry found for ID  tt0089601
No entry found for ID  tt0048233
No entry found for ID  tt0059956
No entry found for ID  tt0028216
No entry found for ID  tt0218864
No entry found for ID  tt0019777
No entry found for ID  tt0116767
No entry found for ID  tt0332285
No entry found for ID  tt0291832
No entry found for ID  tt0342272
No entry found for ID  tt0342258
No entry found for ID  tt0074252
No entry found for ID  tt0104782
No entry found for ID  tt0121766
No entry found for ID  tt0384642
No entry found for ID  tt0369735
No entry found for ID  tt0149261
No entry found for ID  tt0337721
No entry found for ID  tt0055913
No entry found for ID  tt0411705
No entry found for ID  tt0307213
No entry found for ID  tt0393775
No entry found for ID  tt0035790
No entry found for ID  tt0351283
No entry found for ID  tt0381429
No entry found for ID  tt0106233
No entry found for ID  tt0438205
No entry found for ID  tt0449086
No entry f

KeyboardInterrupt: 

Carmen: ID 10-11

In [5]:
get_features(api_key = '4c3b1f8c', ID=10)

number of retrieved IDs:  100
number of retrieved IDs:  200
number of retrieved IDs:  300
number of retrieved IDs:  400
number of retrieved IDs:  500
number of retrieved IDs:  600
number of retrieved IDs:  700
number of retrieved IDs:  800
number of retrieved IDs:  900


In [ ]:
get_features(api_key = '4c3b1f8c', ID=11)

If all features downloaded, aggregated via:

In [6]:
all_omdb_features = pd.DataFrame()
for i in range(0,12):
    all_omdb_features = all_omdb_features.append(pd.read_csv('../data/raw/omdb_'+str(i)+'.csv'))
all_omdb_features.to_csv('../data/raw/omdb_total.csv')

FileNotFoundError: [Errno 2] File ../data/raw/omdb_5.csv does not exist: '../data/raw/omdb_5.csv'